In [68]:
import pandas as pd

In [69]:
import numpy as np

In [157]:
df = pd.read_csv('dados_brutos\Planilha1_2003a2022.csv',sep=';', encoding = 'ISO-8859-1')

In [175]:
df

,data_pag,cpf_servidor,cpf_cnpj_servidor,nome_fornecedor,valor,forma_pag,tipo_despesa,presidente
0,2003-01-02,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,1696.90,D,LOCACAO DE MEIOS DE TRANSPORTE,Luiz Inacio
1,2003-01-02,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,1336.60,D,LOCACAO DE MEIOS DE TRANSPORTE,Luiz Inacio
2,2003-01-02,***.868.251-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,1396.43,D,LOCACAO DE MEIOS DE TRANSPORTE,Luiz Inacio
3,2003-01-03,***.004.131-**,592717000170,FLORES ALVORADA,8585.00,C,OUTROS SERVIOS DE TERCEIROS-PESSOA JURÖDICA,Luiz Inacio
4,2003-01-03,***.004.131-**,839308000125,CRISTAL LIMPEZA - COMRCIO DE MATERIAIS PARA L...,251.50,C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,Luiz Inacio
...,...,...,...,...,...,...,...,...
113335,2022-12-03,***.136.591-**,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,179.50,C,FORNECIMENTO DE ALIMENTAÇO,Jair Bolsonaro
113336,2022-12-03,***.136.591-**,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,54.95,C,GENEROS DE ALIMENTAÇO,Jair Bolsonaro
113337,2022-12-04,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIÇO,518.07,C,GENEROS DE ALIMENTAÇO,Jair Bolsonaro
113338,2022-12-04,***.136.591-**,12550441000146,RESTAURANTE SOHO,330.52,C,FORNECIMENTO DE ALIMENTAÇO,Jair Bolsonaro


In [72]:
# Excluir linhas onde o valor for NaN 
# Converter coluna Data para date type
# Converter coluna CNPJ para string
# Converter coluna valor para float

In [173]:
df.dtypes

data_pag             datetime64[ns]
cpf_servidor                 object
cpf_cnpj_servidor            object
nome_fornecedor              object
valor                       float64
forma_pag                    object
tipo_despesa                 object
dtype: object

## Tratar Dados

In [160]:
# Renomear Colunas
df = df.rename(
    columns= {'DATA PGTO':'data_pag',
             'CPF SERVIDOR':'cpf_servidor',
             'CPF/CNPJ FORNECEDOR':'cpf_cnpj_servidor',
             'NOME FORNECEDOR':'nome_fornecedor',
             'VALOR':'valor',
             'TIPO':'forma_pag',
             'SUBELEMENTO DE DESPESA':'tipo_despesa',
             'CDIC':'cdic'}          
             
            )

In [161]:
# Excluir linhas onde o valor for NaN
df = df.dropna(subset=["valor"])

In [163]:
# Converter coluna Data para date type
df["data_pag"] = pd.to_datetime(df["data_pag"],format="%d/%m/%Y")

In [164]:
# Excluir valores que começam com "-"
# Se retirar o ~ ele deixe só os valores que comneçam com "-"
df = df[~df['valor'].str.startswith('-')]

In [165]:
df["valor"] = df["valor"].str.replace("R$" , "").str.replace("." , "").str.replace("," , ".").replace('  -   ', np.nan).astype(float)
#df["valor"] = df["valor"].astype(float)

In [166]:
# Converter valores não finitos para NaN na coluna 'cpf_cnpj_servidor'
df['cpf_cnpj_servidor'] = df['cpf_cnpj_servidor'].replace([np.inf, -np.inf], np.nan)

# Converter a coluna 'cpf_cnpj_servidor' para o tipo inteiro
df['cpf_cnpj_servidor'] = df['cpf_cnpj_servidor'].astype(float).astype('Int64')

In [167]:
# Converter coluna cpf_cnpj em string
df["cpf_cnpj_servidor"] = df["cpf_cnpj_servidor"].astype(str)

In [168]:
# Excluir coluna cdic
df.drop(['cdic'],axis=1,inplace=True)

In [174]:
# Função personalizada para mapear os valores da coluna "presidente" com base no ano e mês
def mapear_presidente(row):
    ano = row['data_pag'].year
    mes = row['data_pag'].month
    
    if 2003 <= ano <= 2010:
        return 'Luiz Inacio'
    if 2011 <= ano <= 2015:
        return 'Dilma Rousseff'
    if 2017 <= ano <= 2018:
        return 'Michel Temer'
    if 2019 <= ano <= 2022:
        return 'Jair Bolsonaro'
    elif ano == 2016:
        if mes <= 8:
            return 'Dilma Rousseff'
        else:
            return 'Michel Temer'
    

# Aplicar a função ao criar a coluna "presidente"
df["presidente"] = df.apply(mapear_presidente, axis=1)
    

In [176]:
df.to_csv('dados_tratados/cartao_corp_trat.csv', index=False)